### Conduct exeriment

In [40]:
import numpy as np
import time
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.extensions import Initialize
from qiskit_textbook.tools import random_state, array_to_latex

from qiskitflow import Experiment

with Experiment("quantum teleportation") as experiment:
    start_time = time.time()
    
    # conduct experiment as usual
    psi = random_state(1)
    init_gate = Initialize(psi)
    init_gate.label = "init"
    inverse_init_gate = init_gate.gates_to_uncompute()

    qr = QuantumRegister(3, name="q")   # Protocol uses 3 qubits
    crz = ClassicalRegister(1, name="crz") # and 2 classical registers
    crx = ClassicalRegister(1, name="crx")
    qc = QuantumCircuit(qr, crz, crx)
    qc.append(init_gate, [0])
    qc.barrier()
    create_bell_pair(qc, 1, 2)
    qc.barrier()
    alice_gates(qc, 0, 1)
    measure_and_send(qc, 0, 1)
    bob_gates(qc, 2, crz, crx)
    qc.append(inverse_init_gate, [2])
    cr_result = ClassicalRegister(1)
    qc.add_register(cr_result)
    qc.measure(2,2)
    backend_name = "qasm_simulator"
    backend = BasicAer.get_backend(backend_name)
    counts = execute(qc, backend, shots=1024).result().get_counts()
    
    end_time = time.time()
    
    runtime = end_time - start_time
    
    # qiskitflow
    # log parameters used
    experiment.write_parameter("backend name", backend_name)
    # log metrics of experiment
    experiment.write_metric("runtime", runtime)
    # log counts of experiment
    experiment.write_counts("experiment counts", counts)

### Review results

In [41]:
! qiskitflow experiments list


QiskitFlow. Reproducible quantum experiments.

Experiments list

+-----------------------+-------------+
| Name                  |   # of runs |
+=======================+=============+
| quantum teleportation |           3 |
+-----------------------+-------------+


In [42]:
! qiskitflow experiments info "quantum teleportation"


QiskitFlow. Reproducible quantum experiments.

Experiment [quantum teleportation] runs

-- Displayed metrics: all
-- Displayed parameters: all
-- Displayed counts: all
-- Order by metric: NA
-- Order direction: asc

+----------------------------------+-----------------------------+-----------------------------+-----------------------------------------------------------------------------+
| Run ID                           | Parameters                  | Metrics                     | Counts                                                                      |
+==================================+=============================+=============================+=============================================================================+
| 3639ea831dee4bac8e55566517a06589 |                             |                             |                                                                             |
+----------------------------------+-----------------------------+-----------------